In [5]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.applications.densenet import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
%matplotlib inline

In [12]:
total_dataset = pd.read_csv('../datasets/TSLA.csv', index_col=[0], parse_dates=[0])
total_dataset.info()
dataset_train = total_dataset.iloc[:1000]
dataset_train.info()
training_set = dataset_train.iloc[:, 0: 1].values
training_set

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1257 entries, 2017-12-06 to 2022-12-02
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1257 non-null   float64
 1   High       1257 non-null   float64
 2   Low        1257 non-null   float64
 3   Close      1257 non-null   float64
 4   Adj Close  1257 non-null   float64
 5   Volume     1257 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 68.7 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1000 entries, 2017-12-06 to 2021-11-24
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1000 non-null   float64
 1   High       1000 non-null   float64
 2   Low        1000 non-null   float64
 3   Close      1000 non-null   float64
 4   Adj Close  1000 non-null   float64
 5   Volume     1000 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 54.7 KB


array([[ 20.006666],
       [ 20.799999],
       [ 20.973333],
       [ 20.975332],
       [ 22.030001],
       [ 22.728666],
       [ 22.733999],
       [ 22.802668],
       [ 22.993334],
       [ 22.684   ],
       [ 22.179333],
       [ 21.972668],
       [ 21.967333],
       [ 21.588667],
       [ 21.066668],
       [ 20.783333],
       [ 21.078667],
       [ 20.799999],
       [ 21.4     ],
       [ 20.858   ],
       [ 21.108   ],
       [ 21.066668],
       [ 22.344   ],
       [ 22.146667],
       [ 22.349333],
       [ 22.575333],
       [ 22.502666],
       [ 22.698   ],
       [ 23.044666],
       [ 23.      ],
       [ 23.293333],
       [ 24.      ],
       [ 23.638666],
       [ 23.218   ],
       [ 22.766666],
       [ 22.656668],
       [ 23.009333],
       [ 23.167334],
       [ 23.4     ],
       [ 23.229334],
       [ 22.531334],
       [ 21.680668],
       [ 22.599333],
       [ 22.887333],
       [ 21.328667],
       [ 21.075333],
       [ 21.001333],
       [ 21.3

In [6]:
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [6]:
X_train = []
y_train = []
for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60: i, 0])
    y_train.append(training_set_scaled[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)

In [6]:
X_train = np.reshape(X_train, newshape = (X_train.shape[0], X_train.shape[1], 1))

In [6]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(rate = 0.2))
##add 2nd lstm layer
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(rate = 0.2))
##add 3rd lstm layer
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(rate = 0.2))
##add 4th lstm layer
regressor.add(LSTM(units = 50, return_sequences = False))
regressor.add(Dropout(rate = 0.2))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(x = X_train, y = y_train, batch_size = 32, epochs = 100)

In [13]:
dataset_test = total_dataset.iloc[1000:]
dataset_test.info()
real_stock_price = dataset_test.iloc[:, 0: 1].values

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 257 entries, 2021-11-26 to 2022-12-02
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       257 non-null    float64
 1   High       257 non-null    float64
 2   Low        257 non-null    float64
 3   Close      257 non-null    float64
 4   Adj Close  257 non-null    float64
 5   Volume     257 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 14.1 KB


In [ ]:
dataset_total = pd.concat((dataset_train['Open'],dataset_test['Open']), axis = 0)

In [6]:
inputs = dataset_total[len(dataset_total)-len(dataset_test)- 60: ].values
inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)
X_test = []
y_test = []
for i in range(60, len(inputs)):
    X_test.append(inputs[i-60: i, 0])
    X_test = np.array(X_test)
    y_train.append(da[i, 0])
    X_test = np.reshape(X_test, newshape = (X_test.shape[0],
                                            X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)


In [6]:


def smape(A, F):
    return 100 / len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))


def metrics(y_true, y_pred):
    print('R2 Score:', r2_score(y_true=y_true, y_pred=y_pred))
    print('RMSE:', mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False))
    print('MAPE:', mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred))
    return r2_score(y_true=y_true, y_pred=y_pred), mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False), mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred)

In [ ]:
metrics(y_true=)